[Reference](https://blog.stackademic.com/using-celery-rabbitmq-with-fastapi-2e6f0236841e)

```
docker run -p 15672:15672 -p 5672:5672 rabbitmq:3-management
```

In [1]:
!pip install celery python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.0/426.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


In [2]:
# celery_config.py
# Configuration for define celery tasks in a separate file.
import os
from celery import Celery
from dotenv import load_dotenv

load_dotenv() # to load environment variables

celery_app = Celery(__name__, broker=os.getenv("CELERY_BROKER_URL"), backend=os.getenv("CELERY_RESULT_BACKEND"))

celery_app.conf.update(
    imports=['app.tasks.celery_tasks'], # path to your celery tasks file
    broker_connection_retry_on_startup=True,
    task_track_started=True
)

In [3]:
# celery_tasks.py
import asyncio
from app.config.celery_config import celery_app


@celery_app.task
def my_task(x, y):
   ans = x + y
   print(ans)
   return ans


async def my_async_task(x, y):
   await asyncio.sleep(3)
   ans = x + y
   print(ans)
   return ans


@celry_app.task
def my_second_task(x, y):
   result = asyncio.run(my_async_task(x, y))
   return result


In [4]:
# celery_config.py
# Configuration for define celery tasks inside this file.
import os
import asyncio
from celery import Celery
from dotenv import load_dotenv

load_dotenv() # to load environment variables

celery_app = Celery(__name__, broker=os.getenv("CELERY_BROKER_URL"), backend=os.getenv("CELERY_RESULT_BACKEND"))

celery_app.conf.update(
    broker_connection_retry_on_startup=True,
    task_track_started=True
)

@celery_app.task
def my_task(x, y):
   ans = x + y
   print(ans)
   return ans

async def my_async_task(x, y):
   await asyncio.sleep(3)
   ans = x + y
   print(ans)
   return ans

@celry_app.task
def my_second_task(x, y):
   result = asyncio.run(my_async_task(x, y))
   return result

In [5]:
from fastapi import FastAPI

from app.tasks.celery_tasks import my_task

app = FastAPI()


@app.get("/run")
def handle_run():
   task_response = my_task.delay(5, 6)
   return {"message": "Task execution started"}


if __name__ == '__main__':
    uvicorn.run(app, port=8080)

```
celery --app app.config.celery_config.celery_app worker --loglevel=info --pool=solo
```

```
uvicorn app.main:app --port 8000
```